## Backtest Bollinger Bands

- **Indicators:** Bollinger Bands / ATR / Pivot Levels
- **Markets:** spot
- **Position sides:** long / short
- **Timeframe:** 30m

### Strategy long:
- **Open signal:**
    - Price crossing up the bbm or price below bbl
    - Historical price data must show higher highs
    
- **Close signal:**
    - Price crossing downw the bbm or price above bbu
    - Stop loss based on ATR
    
### Strategy short:  
- **Open signal:**
    - Price crossing down the bbm or price above bbu
    - Historical price must have lower lows
    
- **Close signal:**
    - Price crossing up the bbm or price below bbl
    - Stop loss based on ATR
    
    
### Lexicon
- bbl: Bollinger lower band
- bbm: Bollinger medium band
- bbu: Bollinger upper band
- ATR: Average true range
- Pivot levels: Reversal price points



### For better data visualization and analysis, we use Jupyter Notebook to display the different scenarios. The system is using the same code for both the backtesting and live trading.
### Additionally, the analyst does not have to duplicate the code to tweak a strategy 

In [1]:
from datetime import datetime

# Importing python custom modules
from core.Timeframe import Timeframe
from datafeed.other.YahooFinance import YahooFinance
from strategies.BollingerBand import BollingerBand


/usr/local/lib/python3.8/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
# Setting the timeframe
tf = Timeframe('30m')

yf = YahooFinance()
# Pulling the price dataframe from Yahoo finance on the Apple stock for the last month until now on the 30 minutes timeframe
pf = yf.priceframe('AAPL', tf, period= '1mo')

# Backtesting the strategy on the priceframe, assuming we have 10000 as initial capital and each transaction costs 0.2%
backtest = BollingerBand.backtest(pf, cash= 10000, commission= 0.002)
stats = backtest.run()
backtest.plot()

[*********************100%***********************]  1 of 1 completed


Row(id='1469', ...)

### We get valuable information from this simple cross-over strategy to visualize patterns and optimize the parameters.
### Even though the strategy is barely profitable, the result is better than a Buy & Hold strategy, particularly during a downtrend.

In [3]:
print(stats)

Start                     2022-04-26 15:00...
End                       2022-05-26 15:00...
Duration                     30 days 00:00:00
Exposure Time [%]                   29.616725
Equity Final [$]                  9972.801239
Equity Peak [$]                   10156.45284
Return [%]                          -0.271988
Buy & Hold Return [%]               -9.034008
Return (Ann.) [%]                   -2.940014
Volatility (Ann.) [%]                8.258232
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -1.808226
Avg. Drawdown [%]                   -0.344799
Max. Drawdown Duration        9 days 00:00:00
Avg. Drawdown Duration        1 days 08:13:00
# Trades                                   32
Win Rate [%]                           78.125
Best Trade [%]                        9.83243
Worst Trade [%]                     -3.559961
Avg. Trade [%]                    

### We will now see how the same strategy react to another period.

In [4]:
pf = yf.priceframe('AAPL', tf, date_from= datetime.strptime('2022-04-01', '%Y-%m-%d'), date_to= datetime.strptime('2022-05-01', '%Y-%m-%d'))
backtest = BollingerBand.backtest(pf, cash= 10000, commission= 0.002)
stats = backtest.run()
backtest.plot()

[*********************100%***********************]  1 of 1 completed


Row(id='2220', ...)

In [5]:
print(stats)

Start                     2022-04-01 09:30...
End                       2022-04-29 15:30...
Duration                     28 days 06:00:00
Exposure Time [%]                   47.692308
Equity Final [$]                 10138.576663
Equity Peak [$]                  10154.944099
Return [%]                           1.385767
Buy & Hold Return [%]               -8.788476
Return (Ann.) [%]                   18.935107
Volatility (Ann.) [%]                 6.32863
Sharpe Ratio                         2.991976
Sortino Ratio                         7.91868
Calmar Ratio                         12.65741
Max. Drawdown [%]                    -1.49597
Avg. Drawdown [%]                    -0.25759
Max. Drawdown Duration       15 days 21:00:00
Avg. Drawdown Duration        2 days 00:57:00
# Trades                                   23
Win Rate [%]                        82.608696
Best Trade [%]                       8.857638
Worst Trade [%]                     -2.294622
Avg. Trade [%]                    

### In terms of profitability, the result is similar to the previous simulation. However, other metrics, such as the Sharpe and Sortino ratio, changed significantly.
### To conclude the strategy's viability, we must backtest it on multiple periods while avoiding overfitting.

In [6]:
stats_opt = BollingerBand.optimize(
    backtest, 
    bbands_length= range(4, 30, 2), 
    bbands_std= range(1, 5), 
    order_aggreg= range(5, 15),
)
backtest.plot()

/usr/local/lib/python3.8/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 520 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/4 [00:00<?, ?it/s]

Row(id='2971', ...)

### The system defined the best parameters for this period to get the most profitability. However, we can not be sure this will be the case in the future.

In [7]:
print(stats_opt['_strategy'])

BollingerBand(bbands_length=28,bbands_std=2,order_aggreg=11)


In [8]:
print(stats_opt)


Start                     2022-04-01 09:30...
End                       2022-04-29 15:30...
Duration                     28 days 06:00:00
Exposure Time [%]                   52.692308
Equity Final [$]                 10445.017187
Equity Peak [$]                  10445.017187
Return [%]                           4.450172
Buy & Hold Return [%]               -8.788476
Return (Ann.) [%]                   73.083394
Volatility (Ann.) [%]               16.116079
Sharpe Ratio                         4.534812
Sortino Ratio                       24.024823
Calmar Ratio                        46.173833
Max. Drawdown [%]                   -1.582788
Avg. Drawdown [%]                   -0.188325
Max. Drawdown Duration        2 days 23:30:00
Avg. Drawdown Duration        0 days 13:02:00
# Trades                                   38
Win Rate [%]                            100.0
Best Trade [%]                       8.857638
Worst Trade [%]                      0.011556
Avg. Trade [%]                    

### If we want the best profitability with the least exposure time, we can tweak the outcome expectations.

In [9]:
def optim_func(series):
    
    if series['# Trades'] < 5:
        return -1
        
    return series['Equity Final [$]'] / series['Exposure Time [%]']

stats_opt = BollingerBand.optimize(
    backtest, 
    bbands_length= range(4, 30, 2), 
    bbands_std= range(1, 5), 
    order_aggreg= range(5, 15),
    maximize= optim_func,
)
backtest.plot()
print(stats_opt['_strategy'])
print(stats_opt)

/usr/local/lib/python3.8/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 520 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/4 [00:00<?, ?it/s]

BollingerBand(bbands_length=20,bbands_std=3,order_aggreg=7)
Start                     2022-04-01 09:30...
End                       2022-04-29 15:30...
Duration                     28 days 06:00:00
Exposure Time [%]                        20.0
Equity Final [$]                  9988.450645
Equity Peak [$]                  10053.699684
Return [%]                          -0.115494
Buy & Hold Return [%]               -8.788476
Return (Ann.) [%]                    -1.44551
Volatility (Ann.) [%]                2.387014
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.649005
Avg. Drawdown [%]                   -0.314664
Max. Drawdown Duration        7 days 22:30:00
Avg. Drawdown Duration        2 days 06:23:00
# Trades                                    7
Win Rate [%]                        57.142857
Best Trade [%]                       1.962698
Worst Trade [%]     

### We can also use the exact same code to get realtime signals

In [10]:
signals = BollingerBand.live(pf, market= 'spot', cash= 10000, commission= 0.002)
print(signals)

[]
